In [6]:
import pandas as pd
import importlib as il
import sys, os

sys.path.append(os.path.abspath(os.path.join('..', 'modules')))

# project modules
import read as r

In [152]:
data_dir = '../data/'

schema = {
    'countries_data': {
        'extension': '.csv',
        'original_cols': ["Alpha-3 code", "Name", "Population density", "Emission factor tCO2/GWh"],
        'renamed_cols': ["iso3_code", "country_name", "pop_density", "emission_factor"],
        'col_dtypes': ["string", "string", "float64", "float64"]
    },
    'manual_continents_data': {
        'extension': '.csv',
        'original_cols': ["Alpha-3 code", "Name", "Closest continent"],
        'renamed_cols': ["iso3_code", "country_name", "continent_name"],
        'col_dtypes': ["string", "string", "string"]
    }
}

ds_names = list(schema.keys())

In [162]:
# read datasets from files

ds_exts = [schema[key]['extension'] for key in ds_names]
datasets = r.read_datasets(data_dir, ds_names, ds_exts)

In [163]:
# parse datasets

parsed_datasets = {}
v = True # toggle printing dataset info while parsing

for key in ds_names:
    original_cols = schema[key]['original_cols']
    renamed_cols = schema[key]['renamed_cols']
    col_dtypes = schema[key]['col_dtypes']
    print('Dataset: {}'.format(key))
    parsed_datasets[key] = r.parse_datasets(original_cols, renamed_cols, col_dtypes, datasets[key], v=v)
    print('***\n')

Dataset: countries_data
iso3_code           string
country_name        string
pop_density        float64
emission_factor    float64
dtype: object
***

Dataset: manual_continents_data
iso3_code         string
country_name      string
continent_name    string
dtype: object
***



In [170]:
countries_data = parsed_datasets['countries_data']
manual_continents_data = parsed_datasets['manual_continents_data']
countries_data.join(manual_continents_data.set_index('iso3_code')['continent_name'], on='iso3_code')

,iso3_code,country_name,pop_density,emission_factor,continent_name
0,ABW,Aruba,624.00,199.4,<NA>
1,AFG,Afghanistan,48.89,199.4,<NA>
2,AGO,Angola,23.46,199.4,<NA>
3,AIA,Anguilla,140.13,199.4,<NA>
4,ALA,Åland,19.19,199.4,Northern Europe
...,...,...,...,...,...
251,XSP,Spratly Islands,0.00,199.4,Asia
252,YEM,Yemen,63.55,199.4,<NA>
253,ZAF,South Africa,48.14,199.4,<NA>
254,ZMB,Zambia,21.80,199.4,<NA>
